<a href="https://colab.research.google.com/github/kyledinh/gpt-prive/blob/main/notebookds/client-to-hf-inference-endpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -U huggingface_hub python-dotenv

In [ ]:
from huggingface_hub import InferenceClient

# Load .env variables
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

HF_ACCESS_TOKEN = os.getenv("HF_ACCESS_TOKEN", "add-here-if-not-set-in-env-file")
HF_ENDPOINT_URL = os.getenv("HF_ENDPOINT_URL", "add-here-if-not-set-in-env-file")
 
assert HF_ACCESS_TOKEN.startswith("hf_"), "This doesn't look like a valid Hugging Face Token"
print("HF_ACCESS_TOKEN configured")
assert HF_ENDPOINT_URL.startswith("https"), "This doesn't look like a valid Hugging Face Endpoint URL"
print("HF_ACCESS_URL configured")

# Streaming Client
client = InferenceClient(HF_ENDPOINT_URL, token=HF_ACCESS_TOKEN)

# generation parameter
gen_kwargs = dict(
    max_new_tokens=512,
    top_k=30,
    top_p=0.9,
    temperature=0.2,
    repetition_penalty=1.02,
    stop_sequences=["\nUser:", "<|endoftext|>", "</s>"],
)

Non-streaming Example

In [ ]:
# prompt
prompt = "What are good motorcycle riding roads in the United States? Give me 4 examples."
response = client.text_generation(prompt, **gen_kwargs)
print(response)

Prompt example with Streaming

In [ ]:
prompt = "What are good motorcycle riding roads in the United States? Give me 4 examples."

stream = client.text_generation(prompt, stream=True, details=True, **gen_kwargs)

# yield each generated token
for r in stream:
    # skip special tokens
    if r.token.special:
        continue
    # stop if we encounter a stop sequence
    if r.token.text in gen_kwargs["stop_sequences"]:
        break
    # yield the generated token
    print(r.token.text, end = "")
    # yield r.token.text